## Module submission header
### Submission preparation instructions 
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Vishnu Ram Murali
    - Email: vrm32@drexel.edu
- Group member 2
    - Name: Jeromey Abraham
    - Email: ja99@drexel.edu
- Group member 3
    - Name: Mohamed Shehaf Aakil Sharfudeen
    - Email: ms5475@drexel.edu
- Group member 4
    - Name: Jibin Joby
    - Email: jj3225@drexel.edu

### Additional submission comments
- Tutoring support received: NA
- Other (other): NA

# Assignment group 1: Textual feature extraction and numerical comparison

## Module C _(35 points)_ Similarity of word usage across a document

Here we'll be building up some code to discover how different terms are utilized similarly across a document. For this, our first task will be to create a word frequency counting function.

__C1.__ _(12 points)_ Define a function called `count_words(paragraph, pos = True, lemma = True)` that `return`s a `Counter()` called `frequency`. In `frequency`, each key will consist of a `heading = (text, tag)`, where `text` contains the `word.text` attribute from `spacy` if `lemma = False`, and `word.lemma_` attribute if `True`. Similarly, `tag` should be left empty as `""` if `pos = False` and otherwise contain `word.pos_`. The `Counter()` should simply contain the number of times each `heading` is observed in the `paragraph`.

In [1]:
!pip install spacy


In [2]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 12.8 MB 2.8 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import collections

In [4]:
# C1:Function(12/12)

from collections import Counter
import spacy, json, re

nlp = spacy.load("en_core_web_sm")

def count_words(paragraph, pos = True, lemma = True):

    #---Your code starts here
    doc = nlp(paragraph)
    frequency = Counter()
    for word in doc: 
        if lemma:
            text = word.lemma_
        else:
            word.text
        tag = word.pos_ if pos else ""
        frequency[(text, tag)] += 1
    #---Your code ends here
    
    return frequency

Let's make sure your function works by testing it on a short sentence. 

In [5]:
# C1:SanityCheck

count_words("The quick brown fox jumps over the lazy dog.")

Counter({('the', 'DET'): 2,
         ('quick', 'ADJ'): 1,
         ('brown', 'ADJ'): 1,
         ('fox', 'NOUN'): 1,
         ('jump', 'VERB'): 1,
         ('over', 'ADP'): 1,
         ('lazy', 'ADJ'): 1,
         ('dog', 'NOUN'): 1,
         ('.', 'PUNCT'): 1})

__C2.__ _(8 pts)_ Next, define a function called `book_TDM(book_id, pos = True, lemma = True)` and copy into it the TDM-producing code from __Section 2.1.5.1__ of the lecture notes, now `return`-ing `TDM` and `all_words`. Once copied, modify this function to call `count_words` appropriately, now passing through the user of `book_TDM`'s specified `lemma` and `pos` arguments.

In [24]:
# C2:Function(8/8)

import numpy as np
from collections import Counter
import re

def book_TDM(book_id, pos = True, lemma = True):

    #---Your code starts here---
    words = count_words(book_id, pos=pos, lemma=lemma)
    all_words = sorted(set(words)) 
    word_counts = Counter(words)
    n_words = len(all_words)
    TDM = np.zeros((n_words, n_words), dtype=int)

    for i in range(n_words):
        word1 = all_words[i]
        for j in range(i, n_words):
            word2 = all_words[j]
            TDM[i,j] = TDM[j,i] = min(word_counts[word1], word_counts[word2])

    #---Your code ends here---

    return(TDM, all_words)


To test your code's function, let's process `book_id = 84` with both of `pos = True` and `lemma = True` and print out the `TDM`'s `.shape` attribute and the first ten terms in `all_words`.

In [25]:
# C2:SanityCheck

TDM, terms = book_TDM("84", pos = True, lemma = True)
terms[:10]

[('84', 'NUM')]

In [26]:
# C2:SanityCheck

TDM.shape

(1, 1)

__C3.__ _(8 pts)_ Next, your job is to define two functions. The first is `sim(u,v)`, which shoud take two arbitrary numeric vectors and compute/output the `cosine_similarity`, as described in __Section 1.1.2.10__.  

The second function is `term_sims(i, TDM)`, which should utilize the first function (`sim` function) to output a list of cosine similarity values (`sim_values`) between the word/row `i` and all others (rows) in the `TDM`.

Note: each of these functions can be straightforwardly completed using a single line of code! Exhibit your knowledge of comprehensions and vectorization!

In [9]:
# C3:Function(4/8)
def sim(u,v):
    
    #---Your code starts here
    dot_product = sum([x*y for x, y in zip(u, v)])
    magnitude_u = np.sqrt(sum([x**2 for x in u]))
    magnitude_v = np.sqrt(sum([x**2 for x in v]))
    cosine_similarity = dot_product / (magnitude_u * magnitude_v)
    #---Your code ends here
    
    return cosine_similarity

In [10]:
# C3:SanityCheck

print("Exactly similar:", sim(np.array([1,2,3]), np.array([1,2,3])))
print("Exactly dissimilar:", sim(np.array([1,2,3]), np.array([-1,-2,-3])))
print("In the middle:", sim(np.array([1,1]), np.array([-1,1])))

Exactly similar: 1.0
Exactly dissimilar: -1.0
In the middle: 0.0


In [11]:
# C3:Function(4/8)

def term_sims(i, TDM):
    
    #---Your code starts here
    sim_values = []
    u = TDM[i,:]
    for j in range(TDM.shape[0]):
        v = TDM[j,:]
        sim_values.append(sim(u, v))
    #---Your code ends here
    
    return sim_values

In [12]:
# C3:SanityCheck

# Compare word/row 0 to all other (rows) in the TDM
term_sims(0, TDM)

[1.0]

__C4.__ _(7 pts)_ Finally, your goal now is to a write function, `most_similar(term, terms, TDM, top = 25)`, that utilizes `term_sims` to output a sorted list of the `top = N` terms (`top_n_terms`) most similar to one specified (`term`). The output data type should be a list of lists, with each inner list representing information for a similar term as: `[row_ix, similarity, term]`. 

\[Hint: to locate the row containing the term of interest, utilize the list `.index()` method in application to the `terms` argument.\]

In [13]:
# C4:Function(6/7)

def most_similar(term, terms, TDM, top = 25):
    
    #---Your code starts here---
    i = terms.index(term)
    sim_values = term_sims(i, TDM)
    sim_terms = [[j, sim_values[j], terms[j]] for j in range(TDM.shape[0])]
    sim_terms.sort(key=lambda x: x[1], reverse=True)
    top_n_terms = sim_terms[:top]
    
    return top_n_terms

Now, let's test your functions utility on a `TDM` produced for `book_id = 84` and exhibit the top 25 similar terms to both of `('monster', 'NOUN')` and `('beautiful', 'ADJ')`.

In [14]:
# C4:SanityCheck

most_similar(('monster', 'NOUN'), terms, TDM, top = 25)

ValueError: ('monster', 'NOUN') is not in list

In [ ]:
# C4:SanityCheck

most_similar(('beautiful', 'ADJ'), terms, TDM, top = 25)

In [ ]:
# C4:Inline

# Comment on the ordered results returned in the sanity checks.
# Do you think the algorithm is exhibiting sensible results? print "Yes" or "No"
print("No")